<a href="https://colab.research.google.com/github/Olfeng-xalaz/SCM/blob/main/Fallstudie_Juicy_AG_Erweiterung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fallstudie JUICY AG (Network Design)

In [51]:
! pip install -q pyscipopt

In [52]:

import pandas as pd
from pyscipopt import Model, quicksum

## Repo & Daten laden, Bsp. Produktions- und Transportkosten

In [53]:
! git clone https://github.com/AlexKressner/WS25_Supply_Chain_Optimierung

fatal: destination path 'WS25_Supply_Chain_Optimierung' already exists and is not an empty directory.


In [54]:
! ls

sample_data  SCM  WS25_Supply_Chain_Optimierung


In [55]:
! ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign/

NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [56]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

In [57]:
operative_kosten = pd.read_csv(f"{folder}/NetworkDesign_Produktionskosten.csv", sep=";")

In [58]:
operative_kosten.tail()

,Produktionsstandort,Markt,Produktionskosten
20,East London,USA,142
21,East London,Südamerika,100
22,East London,Europa,103
23,East London,Asien,105
24,East London,Afrika,71


In [59]:
operative_kosten.head()

,Produktionsstandort,Markt,Produktionskosten
0,Charleston,USA,81
1,Charleston,Südamerika,92
2,Charleston,Europa,101
3,Charleston,Asien,130
4,Charleston,Afrika,115


In [60]:
kapazitaeten = pd.read_csv(f"{folder}/NetworkDesign_Invest_Kapa.csv", sep=";")

In [61]:
kapazitaeten.head()

,Produktionsstandort,Ausbaustufe,Kapazitäten,Investitionen
0,Charleston,klein,10,6000
1,Charleston,groß,20,9000
2,Curitiba,klein,10,4500
3,Curitiba,groß,20,6750
4,Hamburg,klein,10,6500


In [62]:
nachfrage = pd.read_csv(f"{folder}/NetworkDesign_Nachfrage.csv", sep=";")

In [63]:
nachfrage

,Markt,Nachfragemenge
0,USA,12
1,Südamerika,8
2,Europa,14
3,Asien,16
4,Afrika,7


# Erweiterung

In [64]:
!ls
!ls WS25_Supply_Chain_Optimierung/Daten/NetworkDesign

sample_data  SCM  WS25_Supply_Chain_Optimierung
NetworkDesign_Beschaffungskosten.csv  NetworkDesign_Nachfrage.csv
NetworkDesign_Invest_Kapa.csv	      NetworkDesign_Produktionskosten.csv


In [65]:
folder = "WS25_Supply_Chain_Optimierung/Daten/NetworkDesign"

beschaffung = pd.read_csv(f"{folder}/NetworkDesign_Beschaffungskosten.csv",sep=";")

beschaffung.head()


,Beschaffungsmarkt,Vorprodukt,Beschaffungskosten,Kapazitäten
0,Afrika,Flasche,10,60
1,Afrika,Vitaminwasser,20,30
2,Asien,Flasche,12,100
3,Asien,Vitaminwasser,15,40
4,Europa,Flasche,15,60


# Modell für JUICY AG

In [66]:

# Erstellen einer Modellinstanz
model = Model()

# Indexmengen

In [68]:
I = list(kapazitaeten.Produktionsstandort.unique())
J = list(nachfrage.Markt.unique())
A = list(kapazitaeten.Ausbaustufe.unique())
M = list(beschaffung.Beschaffungsmarkt.unique())
P = list(beschaffung.Vorprodukt.unique())

# Entscheidungsvariablen

In [ ]:
x = {}
for i in I:
  for j in J:
    x[i,j] = model.addVar(vtype='I', name = f"Produktionsmenge am Standort {i} für Markt {j}")

y = {}
for i in I:
  for a in A:
    y[i,a] = model.addVar(vtype='B', name = f"Wenn Werk {i} eine Ausbaustufe {a} baut")

len(model.getVars())


# Parameter

In [ ]:
# Produktions- und Transportkosten
operative_kosten.set_index(["Produktionsstandort","Markt"], inplace=True)
cp = operative_kosten.to_dict("dict")["Produktionskosten"]
print("Produktionskosten:", cp)

#Der Code verwandelt eine Tabelle in einen zweidimensionalen Parameter der direkt im Optimierungsmodell verwendet werden kann. MultiIndex → Tuple-Schlüssel → Modellparamete
#Warum nicht wie im Beispiel davor? Du kannst das so machen – aber nur, wenn dein Parameter eindimensional ist. Hier brauchst du zweidimensionale Parameter.

# Fixkosten
cf = kapazitaeten.set_index(["Produktionsstandort","Ausbaustufe"])
cf = cf.to_dict("dict")["Investitionen"]
print("Investitionen:", cf)


# Kapazitäten
cap = kapazitaeten.set_index("Ausbaustufe")
cap = cap.to_dict("dict")["Kapazitäten"]
print("Kapazitäten:", cap)


# Nachfrage
nachfrage.set_index("Markt", inplace=True)
d = nachfrage.to_dict("dict")["Nachfragemenge"]
print("Nachfragemenge:", d)

# Zielfunktion

In [ ]:
model.setObjective(quicksum(cp[i,j] * x[i,j] for i in I for j in J) + quicksum(cf[i,a] * y[i,a] for i in I for a in A), sense = "minimize")

# Restriktionen

In [ ]:
for j in J: # Produktion des Standorts muss die Nachfrage des Marktes decken
  model.addCons(quicksum(x[i,j] for i in I) >= d[j], name = f' Nachfrage von Markt {j}')

for i in I: # Produktion des Standorts darf logischerweise nicht die Kapazität der Kapazitätsstufe überschreiten
  model.addCons(quicksum(x[i,j] for j in J) <= quicksum(cap[a] * y[i,a] for a in A), name = f'Kapazität vom Standort {i}')

for i in I: # Ein Standort darf nur max. 1 Ausbaustufe besitzen
  model.addCons(quicksum(y[i,a] for a in A) <= 1, name = f'Ausbaustufe der Kapazität{a}')

len(model.getConss())

# Berechnung des Ergebnisses

In [ ]:

model.optimize()
status = model.getStatus()
gap = model.getGap()
print(f"Status des Solvers: {status} , GAP={gap}% \n")

if status == 'optimal':

  print('LÖSUNG:')
  print('Zielfunktionswert (Kosten) =', model.getObjVal()) # Wie hoch sind die minimalen Gesamtkosten?
  print("\n")
  print("PRODUKTIONS-MARKT-ZUTEILUNG") # Wer produziert für wen wie viel (Standort → Markt)?
  for j in J:
    print(f"{j}:")
    for i in I:
      if model.getVal(x[i,j])>0:
        print('\t', f'{i,j} =', round(model.getVal(x[i,j])))
  print("\n")
  print("INVESTITIONEN") # Wo wird investiert (welche Ausbaustufe)?
  for i in I:
    for a in A:
      if model.getVal(y[i,a])>0:
        print(f'{i,a} =', round(model.getVal(y[i,a])))